In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import scipy.optimize as spo

In [124]:
### Markowitz portfolio optimization
class Markowitz:
    """
    Returns the optimal portfolio weights and the corresponding Sharpe ratio.
    Reference: https://ocw.mit.edu/courses/18-s096-topics-in-mathematics-with-applications-in-finance-fall-2013/resources/mit18_s096f13_lecnote14/
    Returns: Must have at least 2 assets each with at least 2 returns listed.
    Return period: >= 2, 12 for monthly, 52 for weekly, 252 for yearly.
    """
    def __init__(self, returns: list, return_period: int):
        self.UNIFORM_WEIGHTS = np.ones(len(returns)) / len(returns)
        self.RETURNS = np.asarray(returns).copy()
        self.FREQUENCY = return_period
        self.COV = np.cov(self.RETURNS)
    
    def _neg_sharpe_ratio(self, weights: list, risk_free_rate: float = 0.0):
        """
        Returns the Sharpe ratio of the portfolio.
        """
        expected_return = np.sum(np.dot(weights, self.RETURNS))*self.FREQUENCY
        standard_deviation = np.sqrt(np.dot(weights, np.dot(self.COV, weights)))*np.sqrt(self.FREQUENCY)
        return (expected_return - risk_free_rate) / standard_deviation
    
    def _variance(self, weights: list):
        """
        Returns the variance of the portfolio.
        """
        return np.dot(weights.T, np.dot(self.COV, weights))

    # Return Maximization
    def max_sharpe(self, risk_free_rate: float = 0.0, weight_constraint: tuple = (0,1)):
        """
        Returns the optimal portfolio weights and the corresponding Sharpe ratio.
        Use the negative Sharpe ratio as the objective function to be minimized.
        """
        init_weights = self.UNIFORM_WEIGHTS
        method='SLSQP'
        bounds=[weight_constraint] * len(self.RETURNS)
        constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        args = (risk_free_rate)
        # minimize the negative Sharpe ratio
        result = spo.minimize(self._neg_sharpe_ratio, init_weights, method=method, bounds=bounds, constraints=constraints, args=args)
        # weights, Sharpe ratio
        return result.x, -result.fun, result.success
        
    # Risk Minimization
    def min_variance(self, weight_constraint: tuple = (0,1)):
        """
        Returns the optimal portfolio weights and the corresponding variance.
        Use the variance as the objective function to be minimized.
        """
        # minimize the variance
        init_weights = self.UNIFORM_WEIGHTS
        method='SLSQP'
        bounds=[weight_constraint] * len(self.RETURNS)
        # , 'type': 'eq', 'fun': lambda x: np.dot(x, self.RETURNS) - target_return
        constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        result = spo.minimize(self._variance, init_weights, method=method, bounds=bounds, constraints=constraints)
        print("result.x", result.x)
        # weights, variance
        return result.x, result.fun, result.success
        


In [127]:
# test max sharpe
returns =[[0.1, 0.2], [0.15 ,0.17], [0.12,0.08], [0.09,0.11], [0.13,0.14]]
markowitz= Markowitz(returns, 2)
weights1, sharpe_ratio, status1 = markowitz.max_sharpe()
print("Status: ", status1)
print('Weights: ', weights1)
print('Sharpe Ratio: ', sharpe_ratio)

# test min variance
weights2, variance, status2 = markowitz.min_variance()
print("Status: ", status2)
print('Weights: ', weights2)
print('Variance: ', variance)


Status:  True
Weights:  [1.00000000e+00 2.73409073e-12 1.68314251e-11 0.00000000e+00
 3.69343445e-12]
Sharpe Ratio:  -6.000000000139668
result.x [0.0297619  0.20436508 0.33531746 0.20436508 0.22619048]
Status:  True
Weights:  [0.0297619  0.20436508 0.33531746 0.20436508 0.22619048]
Variance:  7.688961175772845e-20


In [126]:
x =[[0, 0.2], [0 ,0.17], [0,0.08], [0,0.11], [0,0.14]]
w = [1/len(x)]*len(x)
cov = np.cov(x)
print("cov", cov)
print("w", w)

cov [[0.02    0.017   0.008   0.011   0.014  ]
 [0.017   0.01445 0.0068  0.00935 0.0119 ]
 [0.008   0.0068  0.0032  0.0044  0.0056 ]
 [0.011   0.00935 0.0044  0.00605 0.0077 ]
 [0.014   0.0119  0.0056  0.0077  0.0098 ]]
w [0.2, 0.2, 0.2, 0.2, 0.2]
